In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
import warnings
warnings.filterwarnings('ignore')

ImportError: cannot import name 'LayerNormalization' from 'tensorflow.python.keras.layers.normalization' (C:\Users\nilan\.conda\envs\tf2.6\lib\site-packages\tensorflow\python\keras\layers\normalization\__init__.py)

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [14]:
data = pd.read_csv(r'..\10_fold_cross_validation\train_10folds_208.csv')
data

,MaxPartialCharge,FpDensityMorgan2,BCUT2D_CHGLO,BCUT2D_MRHI,PEOE_VSA12,PEOE_VSA6,SMR_VSA3,SlogP_VSA3,SlogP_VSA8,EState_VSA6,NumHAcceptors,NumSaturatedCarbocycles,fr_bicyclic,TARGET,Kfold
0,0.232165,2.055556,-2.414188,6.375148,5.879988,11.600940,30.133278,11.343745,33.454659,6.196844,10,0,2,1.0,8
1,0.313409,1.155172,-2.211123,7.894021,0.000000,101.109334,4.899910,51.033142,6.076020,62.378411,10,0,0,0.0,9
2,0.394239,1.740741,-2.156618,6.360204,0.000000,17.907236,0.000000,9.589074,0.000000,31.127988,5,0,0,0.0,3
3,0.259309,1.789474,-1.979236,6.347147,5.824404,28.358543,15.124620,0.000000,22.842031,42.725522,4,0,0,0.0,5
4,0.326721,1.804878,-2.225746,9.103009,0.000000,88.440639,9.883888,22.616778,27.751393,12.272864,5,0,2,0.0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734,0.157227,1.928571,-2.406681,7.927002,0.000000,35.334614,9.967957,9.837253,11.257379,24.395945,7,0,0,0.0,6
2735,0.152046,1.958333,-2.386793,6.432995,0.000000,42.258351,9.967957,0.000000,11.257379,12.263211,5,0,0,1.0,2
2736,0.339242,1.900000,-2.136891,6.311472,6.792942,11.600940,0.000000,17.822241,10.969244,32.046576,6,0,2,0.0,5
2737,0.409243,2.000000,-2.396237,7.800537,0.000000,17.667307,14.699729,4.736863,0.000000,4.899910,4,0,0,0.0,8


In [15]:
data['TARGET'].value_counts()

0.0    1771
1.0     968
Name: TARGET, dtype: int64

In [16]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    df = dataframe.iloc[:,:-1].copy()
    labels = df.pop('TARGET')
    df = {key: value[:,tf.newaxis] for key, value in dataframe.iloc[:,:-1].items()}
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(data))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

In [17]:
def get_normalization_layer(name, dataset):
    # Create a Normalization layer for the feature.
    normalizer = layers.Normalization(axis=None)
    
    # Prepare a Dataset that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])
    
    # Learn the statistics of the data.
    normalizer.adapt(feature_ds)
    
    return normalizer

In [18]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    
    # create a layer that turns integer values into integer indices.
    index = layers.IntegerLookup(max_tokens=max_tokens)
    
    # Prepare a `tf.data.Dataset` that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])
    
    # Learn the set of possible values and assign them a fixed integer index.
    index.adapt(feature_ds)
    
    # Encode the integer indices.
    encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())
    
    # Apply multi-hot encoding to the indices. The lambda function captures the
    # layer, so you can use them, or include them in the Keras Functional model later.
    
    return lambda feature: encoder(index(feature))

In [19]:
def train_val_split(fold, batch_size):
    train = data[data.Kfold != fold].reset_index(drop=True)
    valid = data[data.Kfold == fold].reset_index(drop=True)
    train_ds = df_to_dataset(train, batch_size=batch_size)
    val_ds = df_to_dataset(valid, shuffle=False, batch_size=batch_size)
    
    return train_ds, val_ds

In [20]:
def preprocess_dataset(numerical_features, categorical_features):
    all_inputs = []
    encoded_features = []

    # Numerical features.
    for header in numerical_features:
        numeric_col = tf.keras.Input(shape=(1,), name=header)
        normalization_layer = get_normalization_layer(header, train_ds)
        encoded_numeric_col = normalization_layer(numeric_col)
        all_inputs.append(numeric_col)
        encoded_features.append(encoded_numeric_col)
    
    hacc_col = tf.keras.Input(shape=(1,), name='NumHAcceptors', dtype='int64')

    encoding_layer = get_category_encoding_layer(name='NumHAcceptors',
                                                 dataset=train_ds,
                                                 dtype='int64',
                                                 max_tokens=20)
    encoded_hacc_col = encoding_layer(hacc_col)

    all_inputs.append(hacc_col)
    encoded_features.append(encoded_hacc_col)
    
    scc_col = tf.keras.Input(shape=(1,), name='NumSaturatedCarbocycles', dtype='int64')

    encoding_layer = get_category_encoding_layer(name='NumSaturatedCarbocycles',
                                                 dataset=train_ds,
                                                 dtype='int64',
                                                 max_tokens=5)
    encoded_scc_col = encoding_layer(scc_col)

    all_inputs.append(scc_col)
    encoded_features.append(encoded_scc_col)
    
    fbc_col = tf.keras.Input(shape=(1,), name='fr_bicyclic', dtype='int64')

    encoding_layer = get_category_encoding_layer(name='fr_bicyclic',
                                                 dataset=train_ds,
                                                 dtype='int64',
                                                 max_tokens=10)
    encoded_fbc_col = encoding_layer(fbc_col)
    all_inputs.append(fbc_col)
    encoded_features.append(encoded_fbc_col)
    
    return all_inputs, encoded_features

In [21]:
numerical_features = ['MaxPartialCharge', 'FpDensityMorgan2', 'BCUT2D_CHGLO', 'BCUT2D_MRHI',
       'PEOE_VSA12', 'PEOE_VSA6', 'SMR_VSA3', 'SlogP_VSA3', 'SlogP_VSA8', 'EState_VSA6']

categorical_features = ['NumHAcceptors', 'NumSaturatedCarbocycles', 'fr_bicyclic']

In [22]:
class config:
    hidden_layer_1 = 16
    hidden_layer_2 = 64
    hidden_layer_3 = 128
    hidden_layer_4 = 256
    hidden_layer_5 = 32
    hidden_layer_6 = 8
    initializer = tf.keras.initializers.HeNormal()
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    dropout = 0.1
    batch_size = 16
    epochs = 100

# Model Training

In [23]:
def AnnClassifier(all_inputs, encoded_features):
    all_features = tf.keras.layers.concatenate(encoded_features)
    x = tf.keras.layers.Dense(units=config.hidden_layer_1, activation="relu", 
                              kernel_initializer=config.initializer)(all_features)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    #x = tf.keras.layers.Dropout(hparams['dropout'][0])(x)
    x = tf.keras.layers.Dense(units=config.hidden_layer_2, activation="relu",
                             kernel_initializer=config.initializer)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    #x = tf.keras.layers.Dropout(hparams['dropout'][0])(x)
    x = tf.keras.layers.Dense(units=config.hidden_layer_3, activation="relu",
                             kernel_initializer=config.initializer)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    #x = tf.keras.layers.Dropout(hparams['dropout'][0])(x)
    x = tf.keras.layers.Dense(units=config.hidden_layer_4, activation="relu",
                             kernel_initializer=config.initializer)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    x = tf.keras.layers.Dense(units=config.hidden_layer_5, activation="relu",
                             kernel_initializer=config.initializer)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    x = tf.keras.layers.Dense(units=config.hidden_layer_6, activation="relu",
                             kernel_initializer=config.initializer)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(all_inputs, output)
    model.compile(optimizer = config.optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [24]:
def train(fold, all_inputs, encoded_features, train_ds, val_ds, epochs):
    
    print(f"Fold : {fold}")
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
    
    logdir = f'.\\tensorboard_logs\\scalars\\fold_{fold}\\'
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir,write_graph=True,update_freq=1)
    
    checkpoint_filepath = f'.\\model_checkpoints\\fold_{fold}\\'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)
    
    model = AnnClassifier(all_inputs, encoded_features)

    model.fit(train_ds, 
            epochs = epochs, 
            validation_data = val_ds,
            callbacks=[early_stop, tensorboard_callback, model_checkpoint_callback])
    
    model_loss = pd.DataFrame(model.history.history)
    model_loss.to_csv(f".\\Metrics\\metrics_{fold}.csv", index=False)

In [25]:
for fold_ in range(10):
    train_ds, val_ds = train_val_split(fold_, config.batch_size)
    all_inputs, encoded_features = preprocess_dataset(numerical_features, categorical_features)
    train(fold_, all_inputs, encoded_features, train_ds, val_ds, config.epochs)

Fold : 0
Epoch 1/100
155/155 [==============================] - 34s 108ms/step - loss: 0.5346 - accuracy: 0.7432 - val_loss: 0.3845 - val_accuracy: 0.8540


INFO:tensorflow:Assets written to: .\model_checkpoints\fold_0\assets


INFO:tensorflow:Assets written to: .\model_checkpoints\fold_0\assets


Epoch 2/100
155/155 [==============================] - 13s 81ms/step - loss: 0.4152 - accuracy: 0.8243 - val_loss: 0.3359 - val_accuracy: 0.8759


INFO:tensorflow:Assets written to: .\model_checkpoints\fold_0\assets


...

INFO:tensorflow:Assets written to: .\model_checkpoints\fold_9\assets


Epoch 7/100
155/155 [==============================] - 10s 67ms/step - loss: 0.3078 - accuracy: 0.8678 - val_loss: 0.3046 - val_accuracy: 0.9011
Epoch 8/100
155/155 [==============================] - 10s 67ms/step - loss: 0.2986 - accuracy: 0.8808 - val_loss: 0.2865 - val_accuracy: 0.8974
Epoch 9/100
155/155 [==============================] - 11s 68ms/step - loss: 0.2799 - accuracy: 0.8792 - val_loss: 0.2869 - val_accuracy: 0.9084


INFO:tensorflow:Assets written to: .\model_checkpoints\fold_9\assets


INFO:tensorflow:Assets written to: .\model_checkpoints\fold_9\assets


Epoch 10/100
155/155 [==============================] - 10s 66ms/step - loss: 0.2812 - accuracy: 0.8812 - val_loss: 0.2860 - val_accuracy: 0.8864
Epoch 11/100
155/155 [==============================] - 10s 67ms/step - loss: 0.2667 - accuracy: 0.8832 - val_loss: 0.2905 - val_accuracy: 0.8755
Epoch 12/100
155/155 [==============================] - 10s 67ms/step - loss: 0.2629 - accuracy: 0.8885 - val_loss: 0.2955 - val_accuracy: 0.8864
Epoch 13/100
155/155 [==============================] - 10s 64ms/step - loss: 0.2595 - accuracy: 0.8925 - val_loss: 0.2897 - val_accuracy: 0.8864
Epoch 14/100
155/155 [==============================] - 10s 65ms/step - loss: 0.2564 - accuracy: 0.8881 - val_loss: 0.2727 - val_accuracy: 0.8901
Epoch 15/100
155/155 [==============================] - 10s 67ms/step - loss: 0.2653 - accuracy: 0.8852 - val_loss: 0.2466 - val_accuracy: 0.8974
Epoch 16/100
155/155 [==============================] - 10s 65ms/step - loss: 0.2531 - accuracy: 0.8958 - val_loss: 0.2577 -